In [ ]:
# !pip install torch_geometric

Initial baseline update & reward

*   Read current labels from env_work.FrankenData.dist_label.
*   Convert labels to an action matrix with labels_to_action.

Call env_work.step(action_curr). This runs env.step


The returned reward_curr becomes the baseline reward for Metropolis comparisons. This also ensures env_work reflects the post-step opinions (so proposals are compared against a consistent oldX).

In [ ]:
import numpy as np
import torch
# from torch_geometric.data import Data
import math
import pandas as pd


In [ ]:
def labels_to_action(labels, num_districts, dtype=np.float32):
    """
    Convert an integer label vector (shape [N]) to an action matrix
    expected by env.step: shape (N, num_districts), each row is
    a 1-hot encoding of the desired district for that voter.
    """
    N = len(labels)
    A = np.zeros((N, num_districts), dtype=dtype)
    for i, lab in enumerate(labels):
        if lab >= 0 and lab < num_districts:
            A[i, int(lab)] = 1.0
        else:
            # keep row zeros -> will become -1 label in env.step (avoid if possible)
            pass
    return A

#**propose_flip**

The function implements a two-district cluster flip closely following the paper’s steps:

Choose a boundary/adjacent district pair (pick a cut edge).

Construct monochromatic internal edges for those two districts and optionally keep each internal edge with probability p_keep (this implements the Swendsen–Wang style random edge labeling).

Find the connected component (cluster) containing the chosen seed node u.

Flip that whole cluster to the other neighboring district.

Validate the proposed labels (no empty districts, contiguity) and reject if invalid

In [ ]:
import numpy as np

def propose_flip(labels, geo_edge, rng, p_keep):
    """
    Propose a flip following the BFS-from-cut-edge style:
      1. pick random cut edge (u,v)
      2. grow a cluster in one district using BFS with kept mono edges
      3. flip cluster to the other district
      4. reject if it empties a district or breaks contiguity
    """
    N = len(labels)
    labels = labels.astype(int)

    # Step 1: collect cut edges (u,v where districts differ)
    diff_edges = [(int(u), int(v)) for u, v in geo_edge.T if labels[u] != labels[v]]

    if not diff_edges:
        return labels.copy()

    # pick random cut edge
    u, v = diff_edges[rng.integers(len(diff_edges))]
    dA, dB = labels[u], labels[v]

    # Step 2: build adjacency restricted to {dA,dB}, keep edges with prob p_keep
    adj = [[] for _ in range(N)]
    for x, y in geo_edge.T:
        if labels[x] == labels[y] and labels[x] in (dA, dB):
            if rng.random() < p_keep:
                adj[x].append(y)
                adj[y].append(x)

    # BFS cluster starting from u
    cluster = set()
    stack = [u]
    while stack:
        node = stack.pop()
        if node in cluster:
            continue
        cluster.add(node)
        stack.extend(adj[node])

    # Step 3: flip cluster to other district
    cluster_label = labels[u]
    target = dB if cluster_label == dA else dA
    new_labels = labels.copy()
    for node in cluster:
        new_labels[node] = target

    # Step 4a: non-empty check
    if not np.any(new_labels == dA) or not np.any(new_labels == dB):
        return labels.copy()

    # Step 4b: contiguity check
    def is_contiguous(lab):
        nodes = [i for i in range(N) if new_labels[i] == lab]
        if not nodes:
            return False
        seen = {nodes[0]}
        stack = [nodes[0]]
        while stack:
            n = stack.pop()
            for x, y in geo_edge.T:
                if x == n and new_labels[y] == lab and y not in seen:
                    seen.add(y); stack.append(y)
                elif y == n and new_labels[x] == lab and x not in seen:
                    seen.add(x); stack.append(x)
        return len(seen) == len(nodes)

    # only check districts that changed (source & target)
    if not is_contiguous(dA) or not is_contiguous(dB):
        return labels.copy()

    return new_labels



In [ ]:

def simulated_annealing_stage(
    env,
    hot_steps,
    anneal_steps,
    cold_steps,
    T_init,
    T_final,
    rng,
    drf,
    beta1,
    beta2,
    p_keep
):
    """
    Inner loop: run simulated annealing to find the best districting
    for the current environment. Returns best labels, best reward,
    number of accepted proposals, and total proposals.
    """
    curr_labels = env.FrankenData.dist_label.clone().long().numpy().astype(int)
    action_curr = labels_to_action(curr_labels, env.num_districts)
    _, reward_curr, _, _, _ = env.step(action_curr, drf, beta1, beta2)
    best_labels, best_reward = curr_labels.copy(), float(reward_curr)

    total_steps = hot_steps + anneal_steps + cold_steps
    step_idx = 0
    geo_edge = env.FrankenData.geographical_edge.clone().numpy()

    accepted = 0
    proposed = 0

    for phase, n_steps in (("hot", hot_steps), ("anneal", anneal_steps), ("cold", cold_steps)):
        for _ in range(n_steps):
            proposed += 1
            T = max(T_final, T_init + (T_final - T_init) * (step_idx / max(1, total_steps - 1)))
            step_idx += 1

            y_prop = propose_flip(curr_labels, geo_edge, rng, p_keep)
            action_prop = labels_to_action(y_prop, env.num_districts)
            obs_prop, reward_prop, _, _, _ = env.step(action_prop, drf, beta1, beta2)
            diff = float(reward_prop) - float(reward_curr)

            # Metropolis criterion
            if diff > 0:
                p_accept = 1.0
            else:
                p_accept = math.exp(diff / max(1e-8, T))

            if rng.random() < p_accept:
                # Accept proposal
                accepted += 1
                curr_labels, reward_curr = y_prop, float(reward_prop)

                # Track best (even if not accepted)
                if reward_prop > best_reward:
                    best_reward = float(reward_prop)
                    best_labels = y_prop
                    env.commit()
            else:
              if len(env.transition_history) >= 2:
                  env.transition_history.pop()  # remove rejected state
                  env.FrankenData = env.transition_history[-1]


    accept_rate = accepted / max(1, proposed)
    return best_labels, best_reward, accepted, proposed, accept_rate


In [ ]:
import numpy as np, math

def mcmc_full_annealing(
    env,
    outer_loops,
    hot_steps,
    anneal_steps,
    cold_steps,
    T_init,
    T_final,
    seed,
    drf,
    beta1,
    beta2,
    p_keep
):
    initial_mean_opinin = env.FrankenData.opinion.mean().item()
    summary = []
    """
    Outer loop: run full SA search each time, then apply only the best proposal.
    Tracks acceptance statistics from inner loops.
    """
    rng = np.random.default_rng(seed)
    ensemble_labels, ensemble_rewards, ensemble_opinions = [], [], []
    acceptance_rates, total_accepted, total_proposed = [], 0, 0

    for step in range(outer_loops):
        # Inner SA search
        best_labels, best_reward, accepted, proposed, acc_rate = simulated_annealing_stage(
            env, hot_steps, anneal_steps, cold_steps,
            T_init, T_final, rng, drf, beta1, beta2, p_keep
        )

        total_accepted += accepted
        total_proposed += proposed
        acceptance_rates.append(acc_rate)

        # Apply only the best found configuration to the environment
        best_action = labels_to_action(best_labels, env.num_districts)
        obs_best, reward_best, _, _, _ = env.step(best_action, drf, beta1, beta2)

        ensemble_labels.append(best_labels)
        ensemble_rewards.append(float(reward_best))
        ensemble_opinions.append(obs_best.opinion)

        ops = [x[0].item() for x in obs_best.opinion]

        summary.append({
            "step": step,
            'initial_mean_opinin': initial_mean_opinin,
            'last_mean_opinin': obs_best.opinion.mean().item(),
            'best_labels' : best_labels,
            'reward_best': reward_best,
            'best_opinion': ops
        })
        df = pd.DataFrame(summary)

        print(f"[Outer step {step+1}/{outer_loops}] best reward = {reward_best:.4f}, "
              f"inner acceptance rate = {acc_rate:.3f}")

    global_accept_rate = total_accepted / max(1, total_proposed)
    print(f"\nTotal proposed: {total_proposed}, accepted: {total_accepted}, "
          f"overall acceptance rate: {global_accept_rate:.3f}")

    return (df,env, ensemble_labels, ensemble_rewards,
            ensemble_opinions, acceptance_rates, global_accept_rate)